# Crawl the Wiki

In [3]:
import os
import glob
import time
import json
import random
import tensorflow as tf
import requests
import numpy as np
from tqdm import tqdm 
from bs4 import BeautifulSoup
import re
from collections import Counter

In [8]:
class crawling:
    def __init__(self, root_link, crawl_level):
        self.root = root_link
        self.level = crawl_level
        self.root_dir = ''
    def run(self):
        self.root_dir = self.creat_dir()
        link_list = [self.root]
        for level in range(0,self.level):  
            level_links = []
            level_para = ''
            print("\n Level ",str(level))
            for link in tqdm(link_list):
                link_list, para = self.crawl_link(link) 
                level_links += link_list
                level_para += para
            f = open(os.path.join(self.root_dir, "level"+str(level)+".txt"),"w+")
            f.write(level_para)
            f.close()
            link_list = level_links
            time.sleep(15)

        filenames = []
        for file in glob.glob(self.root_dir+"/*.*"):
            filenames.append(file.split("/")[1])
        if 'levels.txt' in filenames:
            filenames.remove('levels.txt')
  
        with open(os.path.join(self.root_dir, "levels.txt"), 'w') as outfile:
            for fname in filenames:
                with open(os.path.join(self.root_dir, fname)) as infile:
                    for line in infile:
                        outfile.write(line) 
    def creat_dir(self):
        temp = ''
        for char in self.root:
            if char in [':', '/', '.']:
                temp += '_'
            else: 
                temp += char
        try:
            os.mkdir(temp)
        except OSError:  
            print("Directory Exists!")
        return temp
    def remove_tags(self, p):
        p = str(p)
        temp = ''
        tagflag = 0
        for char in p:
            if char == "<":
                tagflag = 1
            if char == ">":
                tagflag = 0
                continue
            if tagflag == 0:
                temp += char 
        return temp
    def remove_bm(self, p):
        p = str(p)
        temp = ''
        tagflag = 0
        for char in p:
            if char == "[":
                tagflag = 1
            if char == "]":
                tagflag = 0
                continue
            if tagflag == 0: 
                temp += char 
        return temp
    def remove_brak(self, p):
        p = str(p)
        temp = ''
        tagflag = 0
        for char in p:
            if char == "(":
                tagflag = 1
            if char == ")":
                tagflag = 0
                continue
            if tagflag == 0: 
                temp += char 
        return temp
    def remove_num(self, p):
        p = str(p)
        temp = '' 
        for char in p:
            if char in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
                temp += ''
            else:
                temp += char
        return temp
    def edit_links(self, link_list):
        for i in range(len(link_list)-1 , -1 , -1 ):
            if link_list[i] is None:
                link_list.remove(link_list[i])

        for i in range(len(link_list)-1 , -1 , -1 ):
            if 'wiki' not in str(link_list[i]):
                link_list.remove(link_list[i])
        for i in range(0, len(link_list)):
            if str(link_list[i][:6]) == '/wiki/':
                link_list[i] = 'https://en.wikipedia.org' + link_list[i]
        #filter wiki tools
        for i in range(len(link_list)-1 , -1 , -1 ):
            if str(link_list[i][:2]) == '//':
                link_list.remove(link_list[i])
        #english pages
        for i in range(len(link_list)-1 , -1 , -1 ):
            if 'https://en.' not in str(link_list[i]):
                link_list.remove(link_list[i])
        #Filter Images Links
        for i in range(len(link_list)-1 , -1 , -1 ):
            if '.jpg' in str(link_list[i]) or '.png' in str(link_list[i]) or '.jpeg' in str(link_list[i]):
                link_list.remove(link_list[i])
        return link_list
    def crawl_link (self, link):
        links = []
        paragraph = ''
        root = requests.get(link)
        if root.status_code == 200:
            #Code
            #<div id="bodyContent" class="mw-body-content">
            soup = BeautifulSoup(root.content)
            samples = soup.find_all("div", "mw-body-content")
            #get all links in the page 
            for l in soup.find_all('a'):
                links.append(l.get('href'))
            links = self.edit_links(links)
            #get all paragraphs
            parags = soup.find_all('p')
            #extract paragraphs without html tags
            for i in range(0,len(parags)):
                paragraph += self.remove_brak(self.remove_num(self.remove_bm(self.remove_tags(parags[i]))))
        else:
            print("Page Not Found!")
        return links, paragraph

In [10]:
crawl = crawling("https://en.wikipedia.org/wiki/Dentistry",2)
crawl.run()

  0%|          | 0/1 [00:00<?, ?it/s]

Directory Exists!

 Level  0


/home/mgd-pc/.local/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/mgd-pc/anaconda3/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
  0%|          | 0/637 [00:00<?, ?it/s]


 Level  1


100%|██████████| 637/637 [18:59<00:00,  1.79s/it]


# Embeddings

In [17]:
def preprocessU(text):
    # Replace punctuation with tokens so we can use them in our model
    text = text.lower()
    text = text.replace('.', ' <PERIOD> ')
    text = text.replace(',', ' <COMMA> ')
    text = text.replace('"', ' <QUOTATION_MARK> ')
    text = text.replace(';', ' <SEMICOLON> ')
    text = text.replace('!', ' <EXCLAMATION_MARK> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    text = text.replace('(', ' <LEFT_PAREN> ')
    text = text.replace(')', ' <RIGHT_PAREN> ')
    text = text.replace('--', ' <HYPHENS> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    # text = text.replace('\n', ' <NEW_LINE> ')
    text = text.replace(':', ' <COLON> ')
    words = text.split()
    
    # Remove all words with  5 or fewer occurences
    word_counts = Counter(words)
    trimmed_words = [word for word in words if word_counts[word] > 5]

    return trimmed_words

def get_batchesU(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: A list where each item is a tuple of (batch of input, batch of target).
    """
    n_batches = int(len(int_text) / (batch_size * seq_length))

    # Drop the last few characters to make only full batches
    xdata = np.array(int_text[: n_batches * batch_size * seq_length])
    ydata = np.array(int_text[1: n_batches * batch_size * seq_length + 1])

    x_batches = np.split(xdata.reshape(batch_size, -1), n_batches, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), n_batches, 1)

    return list(zip(x_batches, y_batches))


def create_lookup_tablesU(words):
    """
    Create lookup tables for vocabulary
    :param words: Input list of words
    :return: A tuple of dicts.  The first dict....
    """
    word_counts = Counter(words)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab

def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    R = np.random.randint(1, window_size+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    target_words = set(words[start:idx] + words[idx+1:stop+1])
    return list(target_words)


def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    n_batches = len(words)//batch_size
    # only full batches
    words = words[:n_batches*batch_size]
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y

In [12]:
with open(os.path.join(crawl.root_dir, "levels.txt")) as f:
    text = f.read()
words = preprocessU(text)
print(words[:30])

['dentistry', '<COMMA>', 'also', 'known', 'as', 'dental', 'and', 'oral', 'medicine', '<COMMA>', 'is', 'a', 'branch', 'of', 'medicine', 'that', 'consists', 'of', 'the', 'study', '<COMMA>', 'diagnosis', '<COMMA>', 'prevention', '<COMMA>', 'and', 'treatment', 'of', 'diseases', '<COMMA>']


In [13]:
print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

Total words: 1370744
Unique words: 12885


In [14]:
vocab_to_int, int_to_vocab = create_lookup_tablesU(words)
int_words = [vocab_to_int[word] for word in words]

In [15]:
threshold = 1e-5
word_counts = Counter(int_words)
total_count = len(int_words)
freqs = {word: count/total_count for word, count in word_counts.items()}
p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts}
train_words = [word for word in int_words if random.random() < (1 - p_drop[word])]

In [18]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32, [None], name='inputs')
    labels = tf.placeholder(tf.int32, [None, None], name='labels')

In [19]:
n_vocab = len(int_to_vocab)
n_embedding = 300 # Number of embedding features 
with train_graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_vocab, n_embedding), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs)

In [20]:
# Number of negative labels to sample
n_sampled = 100
with train_graph.as_default():
    softmax_w = tf.Variable(tf.truncated_normal((n_vocab, n_embedding), stddev=0.1))
    softmax_b = tf.Variable(tf.zeros(n_vocab))
    
    # Calculate the loss using negative sampling
    loss = tf.nn.sampled_softmax_loss(softmax_w, softmax_b, 
                                      labels, embed,
                                      n_sampled, n_vocab)
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [21]:

with train_graph.as_default():
    ## From Thushan Ganegedara's implementation
    valid_size = 16 # Random set of words to evaluate similarity on.
    valid_window = 100
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, 
                               random.sample(range(1000,1000+valid_window), valid_size//2))

    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
    normalized_embedding = embedding / norm
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [22]:
# If the checkpoints directory doesn't exist:
!mkdir checkpoints

In [ ]:
epochs = 25
batch_size = 1000
window_size = 5

with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())

    for e in range(1, epochs+1):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        for x, y in batches:
            
            feed = {inputs: x, labels: np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            loss += train_loss
            
            if iteration % 100 == 0: 
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
            
            if iteration % 1000 == 0:
                # note that this is expensive (~20% slowdown if computed every 500 steps)
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k = 8 # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = int_to_vocab[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)
            
            iteration += 1
    save_path = saver.save(sess, "checkpoints/text8.ckpt")
    embed_mat = sess.run(normalized_embedding)

In [ ]:
with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    embed_mat = sess.run(embedding)

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
viz_words = 2500
tsne = TSNE()
embed_tsne = tsne.fit_transform(embed_mat[:viz_words, :])

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label

%time
viz_words = 1000
tsne = TSNE()
embed_tsne = tsne.fit_transform(embed_mat[:viz_words, :])

output_notebook()
desc = []
for idx in range(viz_words):
    desc.append(int_to_vocab[idx])
    
list_x = list(embed_tsne[0:viz_words, 0])
list_y = list(embed_tsne[0:viz_words, 1])

source = ColumnDataSource(data=dict(x=list_x, y=list_y, desc=desc)) 
hover = HoverTool(tooltips=[
    ('desc', '@desc'),
])
mapper = LinearColorMapper(palette=plasma(256), low=min(list_y), high=max(list_y))

p = figure(plot_width=1000, plot_height=1000, tools=[hover,"pan,wheel_zoom,box_zoom"], 
           title="Word2Vec")
p.circle('x', 'y', size=10, source=source)

labels = LabelSet(x='x', y='y', text='desc', level='glyph',
              x_offset=5, y_offset=5, source=source, render_mode='canvas')

p.add_layout(labels)
show(p)